In [ ]:
import cv2, matplotlib
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame, read_csv
import pandas as pd

import Image
import math
from PIL import Image
from PIL import ImageStat
import os.path

In [ ]:
def brightness( im_file ):
   im = Image.open(im_file)
   status = ImageStat.Stat(im)
   r,g,b = status.rms
   return math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

In [ ]:
def rgb_to_hsv(input):
    image = cv2.imread(input, cv2.IMREAD_COLOR)  # uint8 image
    norm_image = cv2.normalize(image, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    return matplotlib.colors.rgb_to_hsv(norm_image)

In [ ]:
def hist_gram_blue(input):
    imgg = cv2.imread(input)
    return cv2.calcHist([imgg],[0],None,[256],[0,256])

In [ ]:
def hist_gram_green(input):
    imgg = cv2.imread(input)
    return cv2.calcHist([imgg],[1],None,[256],[0,256])

In [ ]:
def hist_gram_red(input):
    imgg = cv2.imread(input)
    return cv2.calcHist([imgg],[2],None,[256],[0,256])

In [ ]:
#define input output csv files
inputcsv = 'ceramic_pmg_nnegev.csv'
outputcsv='output.csv'

#exception handling
if(outputcsv.find('.csv') == -1):
    raise ValueError('Output has to be a csv file. It has to end with .csv. Abort ')
if(inputcsv.find('.csv') == -1):
    raise ValueError('Input has to be a csv file. It has to end with .csv. Abort ')

#open csv and get the list of images
df=pd.read_csv(inputcsv)
image_list=df['#img'].tolist()

In [ ]:
#initialize separate lists
bright_list=[]
hsv_list=[]
histogram_blue_list=[]
histogram_green_list=[]
histogram_red_list=[]


#calculate different properties
for x in image_list:
    img='images/'+x+'.jpg'
    #check to make sure it is a valid file
    if(not os.path.exists(img)):
        raise ValueError('The file does not exist. Please recheck your list in csv. Abort.')
    bright_list.append(brightness(img))
    hsv_list.append(rgb_to_hsv(img))
    histogram_blue_list.append(hist_gram_blue(img))
    histogram_green_list.append(hist_gram_green(img))
    histogram_red_list.append(hist_gram_red(img))

brt = pd.DataFrame({'brightness': bright_list})
hsv_l=pd.DataFrame({'hue_saturation_value': hsv_list})
hist_b=pd.DataFrame({'histogram_blue': histogram_blue_list})
hist_g=pd.DataFrame({'histogram_green': histogram_green_list})
hist_r=pd.DataFrame({'histogram_red': histogram_red_list})


#concatenate properties to original dataframe
result = pd.concat([df, brt,hsv_l,hist_b,hist_g,hist_r], axis=1)

#result.head()

In [ ]:
#output file
result.to_csv(outputcsv, index = False)